In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.simplefilter("ignore")

In [3]:
prishvin = pd.read_csv("http://r.senyukhin.ru/files/prishvin.csv", sep='\t')
prishvin.head()

,id,text,date,dateTop,notDated,julian_calendar
0,17938,"<p>Вчера был наш русский Новый год, он старыми...",1914-01-14,0000-00-00,0,1
1,17939,<p>Зимний рассвет: строгий красный начался и н...,1914-01-19,0000-00-00,0,1
2,17940,"<p>Зимой рано утром, когда начинает светать, с...",1914-01-25,0000-00-00,0,1
3,17941,<p>Наблюдения в Песочках. Почтальон Николай бы...,1914-01-26,0000-00-00,0,1
4,17942,<p>Собрание Религиозно-философского общества д...,1914-02-01,0000-00-00,0,1


### Этап 1. Очистка данных

In [10]:
# Удаление тегов и следов маркдауна
patterns = [r'<com id=\d+"/>', r'<com id="\d+"/>', r'<\w+>', r'<\/\w+>', r'<\w+\s/>', '\*', '#']

for pattern in patterns:
    prishvin['text'] = prishvin['text'].str.replace(pattern, ' ', regex=True)

# Удаление следов плохой типографики
prishvin['text'] = prishvin['text'].str.replace('(\w+-)\s([го|я|й])', '$1$2', regex=True)

# Я решил удалить следы раскрытия сокращений слов, например, М[ария]
prishvin['text'] = prishvin['text'].str.replace('(\w)\[(\w+)\]', '$1$2', regex=True)


### Этап 2. Предобработка данных
Поскольку мы хотим изучить тенденцию частотности слов по годам, то я создам колонку с годом записи

In [11]:
prishvin['year'] = pd.to_datetime(prishvin['date'], format='%Y-%m-%d').dt.year
years = prishvin['year'].unique()


In [95]:
# Для тренировки построю модель для всего дневника
vectorizer = TfidfVectorizer(use_idf=True)
X = vectorizer.fit_transform(prishvin['text'])
prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
prishvin_words.head(10)

,TF-IDF
разбойники,0.199611
покаялись,0.193081
бебеля,0.162729
злодеяний,0.132920
хулиганов,0.122802
хулиганство,0.120552
монастыри,0.112685
крестьянская,0.108486
крещение,0.105229
видели,0.104275


### Этап 3. Исследование по годам

In [12]:
years.sort()

prishvin_by_years = pd.DataFrame()

In [ ]:
# Первое приближение 
for year in years:
    vectorizer = TfidfVectorizer(use_idf=True)
    X = vectorizer.fit_transform(prishvin[prishvin['year'] == year]['text'])
    prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
    prishvin_by_years[year] = prishvin_words.head(10).index

In [92]:
prishvin_by_years

,1905,1906,1908,1909,1910,1911,1912,1913,1914,1915,...,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954
0,не,не,сознания,письмо,он,зотов,не,не,не,сделала,...,роли,племянница,поэт,кто,мылся,траур,правда,друг,стойло,лучшими
1,но,нет,жизни,что,на,фокусы,конец,что,свет,завещание,...,личности,валяюсь,пришел,на,шаховы,письмо,бразная,пятницы,баранов,годом
2,бы,теперь,должно,не,не,трудовик,что,жизни,то,его,...,истории,встречал,буи,портрет,вчерашняя,что,подтвердила,хоронить,важнее,сидел
3,ее,на,жизнь,вдруг,так,жена,но,могу,как,видимому,...,заявка,визитом,ствовал,распад,брился,не,образах,проболтали,год,пожеланиями
4,что,все,для,написал,что,душе,когда,помню,разбойники,выстроитесь,...,требование,метелицей,бешено,борется,спеша,романа,шилькреды,простужусь,стойлу,весной
5,мне,он,что,было,все,так,на,мария,на,ожила,...,сережа,готовит,вернулись,попробовал,болтали,меньше,воплощение,барвихи,стойлом,пантелеймону
6,она,это,назвал,письма,то,от,любовь,дв,что,наследников,...,что,эстонии,удинцевым,не,горячая,ему,мечта,среда,бараны,писателя
7,на,то,даты,меня,его,нельзя,мысли,ивановна,год,начала,...,нравственного,чашу,симпатичная,простой,серая,год,поели,отправляется,время,принимал
8,от,бог,вечной,то,говорит,обманщики,так,послать,бебеля,парк,...,себе,мягкая,лепесток,человек,ни,так,мечтал,будками,году,месте
9,кажется,никогда,ругали,ее,за,хе,все,но,покаялись,случайные,...,право,возобновился,соломенную,свете,весело,моего,звезд,охранными,когда,кондратьевичу


Результаты содержат много [стоп-слов](https://ru.wikipedia.org/wiki/Шумовые_слова). Предлагаю от них избавиться.

Я взял список стоп-слов отсюда: https://github.com/stopwords-iso/stopwords-ru/blob/master/stopwords-ru.txt?ysclid=ldoas1im8w726822300

In [13]:
import requests

data = requests.get('http://r.senyukhin.ru/files/stopwords-ru.txt')
stop_words = data.text.split('\n')

for year in years:
    vectorizer = TfidfVectorizer(use_idf=True, stop_words=stop_words)
    X = vectorizer.fit_transform(prishvin[prishvin['year'] == year]['text'])
    prishvin_words = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"]).sort_values('TF-IDF', ascending=False)
    prishvin_by_years[year] = prishvin_words.head(20).index


In [14]:
prishvin_by_years.loc[:, '1920':'1930']

,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930
0,раздирает,парижский,ценз,маркса,рукопись,судьба,трактир,алпатов,снизу,летят,спустят
1,товарами,достоевск,не,лев,руко,судьбу,что,что,окна,засыпаны,колокол
2,оставленных,подросток,что,вагоне,рукописью,жизни,провинции,разве,снегом,пороши,известно
3,приказчик,средняя,личность,социализм,мне,творчества,нее,франс,наружных,наверное,ней
4,железным,романа,кто,вот,она,дело,деле,генерала,пекину,леса,мотиве
5,отчее,любили,потому,он,не,меня,не,оболочка,намекая,каждый,носы
6,далекую,чтения,вождь,львов,пись,вызывает,вас,социализм,уютнее,день,богатствах
7,ситец,бы,спеца,дамочка,на,вызываешь,кого,человечеству,морозных,уже,упавший
8,ситцев,любила,кумир,против,руку,ба,свет,учение,подбило,очень,ервого
9,ситцы,меньше,бы,талдом,беда,время,ничего,не,узорах,проводить,еликой


Вообще примечательно, что год от года слова практически не повторяются, а часть этих лемм — это разные формы. 